<a href="https://colab.research.google.com/github/yellayujin/mulcamp34/blob/main/240116_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Table of Contents
- 0.[Outline](#ch0)
- 1.[Data Exploration](#ch1)
    - 1.1[Exploratory Data Analysis](#ch1.1)
    - 1.2[Churn Age Analysis](#1.2)
- 2.[Detailed Plan Establishment](#ch2)
    - 2.1[Deeper EDA](#ch2.1)
- 3.[Statistical Test](#ch3)
    - 3.1[Sampled T-test](#ch3.1)
    - 3.2[chi-square test](#ch3.2)
- 4.[Regression](#ch4)
- 5.[Conclusion](#ch5)
- 6.[Project Evaluation](#ch6)

### All the used libraries:
- pandas: 2.14
- numpy: 1.26.2
- matplotlib:3.8.0
- seaborn: 0.12.2
- scipy: 1.11.4
- sklearn: 1.3.2
- statsmodels: 0.14.0

# 0. Outline<a class='anchor' id='ch0'></a>
- This dataset was generated from a deep learning model trained on the Bank Customer Churn Prediction dataset. Feature distributions are close to, but not exactly the same, as the original.
- The analysis is part of practicing skills learned in the 'Data Analysis & Engineering Camp (Python)'.; therefore, essential steps may have been omitted, given the focus on Exploratory Data Analysis (EDA) in 3 days.
- As we are going to focus on EDA(Exploritory Plan Analysis), we'll look through the trian.csv which includes the target variable, 'Exited'.


### BackGround
- The recent increase in bank customer churn has created a number of problems.
- In order to minimize customer churn and increase customer retention, we are going to identify and solve various problems and even attract new customers.
- In order to identify the cause of the customer churn, we will analyze various columns and identify the problem, identify which part is the problem, and solve and supplement the problem.


# 1. Data Exploration <a class='anchor' id='ch1'></a>

### Data Description
- **Customer ID**: A unique identifier for each customer
- **Surname**: The customer's surname or last name
- **Credit Score**: A numerical value representing the customer's credit score
- **Geography**: The country where the customer resides
- **Gender**: The customer's gender
- **Age**: The customer's age.
- **Tenure**: The number of years the customer has been with the bank
- **Balance**: The customer's account balance
- **NumOfProducts**: The number of bank products the customer uses (e.g., savings account, credit card)
- **HasCrCard**: Whether the customer has a credit card
- **IsActiveMember**: Whether the customer is an active member
- **EstimatedSalary**: The estimated salary of the customer
- **Exited**: Whether the customer has churned (Target Variable)

In [ ]:
import pandas as pd
train = pd.read_csv('/kaggle/input/train-csv/train.csv')
train.info()

- No null data.

In [ ]:
# Descriptive statistics for all data
continuous_var = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
train[continuous_var].describe().T

In [ ]:
# Descriptive statistics of churn
train.loc[train['Exited']==1,continuous_var].describe().T

### 1.1 Exploratory Data Analysis<a class='anchor' id='ch1.1'></a>
- Before selecting our target audience, we conducted an EDA of our data.

In [ ]:
# Number of customers per country
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(5, 3))
ax = sns.countplot(x='Geography', data=train, palette='viridis')

In [ ]:
# Customer distribution by gender
plt.figure(figsize=(5, 3))
sns.countplot(x='Gender', data=train, palette='Set1')
plt.title('Users by Gender')
plt.xlabel('Gender')
plt.show()

In [ ]:
# Customer distribution by age
plt.figure(figsize=(5, 3))
sns.histplot(train['Age'], bins=30, color='skyblue')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.show()

In [ ]:
# Distribution of churn by age
exited = train.loc[train['Exited']==1,:]

plt.figure(figsize=(8, 6))
sns.histplot(x='Age', data=exited, bins=30)
plt.title('Exited Distribution')
plt.xlabel('Exited')
plt.show()

In [ ]:
# Credit score distribution
plt.figure(figsize=(5, 3))
sns.histplot(train['CreditScore'], bins=30)
plt.title('CreditScore')
plt.xlabel('CreditScore')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Tenure distribution
plt.figure(figsize=(5, 3))
sns.countplot(x='Tenure', data=train)
plt.xlabel('Tenure (years)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Balance distribution
plt.figure(figsize=(5, 3))
sns.histplot(train['Balance'], bins=30, kde=True)
plt.xlabel('Balance')
plt.ylabel('Frequency')
plt.show()

- There are a significant number of customers with a Balance of 0 (45.67% of the total), which may affect the analysis, so we exclude them.

In [ ]:
# Data excluding zero balances
data = train.drop(['id', 'CustomerId', 'Surname', 'Geography'], axis=1)

data_ybal = data.loc[data['Balance'] != 0, :]
data.shape[0]
data_ybal.shape[0]

plt.figure(figsize=(5, 3))
sns.histplot(data_ybal['Balance'], bins=30)
plt.title('non-zero Balance')
plt.xlabel('Balance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Distribution of the number of products used
plt.figure(figsize=(5, 3))
sns.countplot(x='NumOfProducts', data=train)
plt.xlabel('NumOfProducts')
plt.ylabel('Count')
plt.show()


In [ ]:
# Distribution of customers with credit cards
sns.countplot(x='HasCrCard', data=train, palette='cool')

plt.grid(axis='y')
plt.xlabel('HasCrCard', fontsize=13)
plt.xticks([0, 1], ['N', 'Y'])
plt.show()

In [ ]:
# Active member distribution
sns.countplot(x='IsActiveMember', data=train, palette='cool')

plt.grid(axis='y')
plt.xlabel('IsActiveMember', fontsize=13)
plt.xticks([0, 1], ['N', 'Y'])
plt.show()

- We can analyze the relationship with other variables to understand which factors influence customer activation.
- We also can use this information to create reward programs for loyal customers or run reactivation campaigns for inactive members.

In [ ]:
# Number of churned members
plt.figure(figsize=(5, 3))
sns.countplot(x='Exited', data=data)
plt.xlabel('Exited', fontsize=13)
plt.xticks([0, 1], ['N', 'Y'])
plt.ylabel('Count')
plt.show()

- Leavers are 1/4 the size of existing members

In [ ]:
# Estimated salary distribution
# Show in thousands
train.loc[:, 'EstimatedSalary_thousands'] = train['EstimatedSalary'] / 1000

salary_bins = pd.cut(train['EstimatedSalary_thousands'], bins=[0,20,40,60,80,100,120,140,160,180,200])
salary_bin_counts = salary_bins.value_counts().sort_index()

plt.figure(figsize=(7, 4))
ax = sns.barplot(x=salary_bin_counts.index.astype(str), y=salary_bin_counts.values, palette='cool')

plt.xticks(rotation=45)
plt.xlabel('EstimatedSalary(k)', fontsize=13)
plt.grid(axis='y')
plt.show()

### 1.2 Churn Age Analysis <a class='anchor' id='ch1.2'></a>
- We decided that if we detected significant churn in a particular age group, we would target that age group.

In [ ]:
# Calculate churn rate by age group
def create_age_group(age):
    if age < 20:
        return '10s'
    elif age < 30:
        return '20s'
    elif age < 40:
        return '30s'
    elif age < 50:
        return '40s'
    elif age < 60:
        return '50s'
    elif age < 70:
        return '60s'
    else:
        return '70s up'

train['AgeGroup'] = train['Age'].apply(create_age_group)

AgeGroup = ['10s', '20s', '30s', '40s', '50s', '60s', '70s up']
for i in AgeGroup:
    AgeExited = train.loc[(train['AgeGroup'] == i) & (train['Exited'] == 1) , :].shape[0]
    AgeData = train.loc[train['AgeGroup'] == i].shape[0]
    print(i, round(AgeExited / AgeData, 3))

In [ ]:
# Visualize churn rate by age group
AgeGroup = ['10s', '20s', '30s', '40s', '50s', '60s', '70s up']
AgeExitRates = []

for i in AgeGroup:
    AgeExited = train.loc[(train['AgeGroup'] == i) & (train['Exited'] == 1) , :].shape[0]
    AgeData = train.loc[train['AgeGroup'] == i].shape[0]
    exit_rate = round(AgeExited / AgeData, 3) if AgeData > 0 else 0
    AgeExitRates.append(exit_rate)


plt.figure(figsize=(7, 4))
plt.bar(AgeGroup, AgeExitRates, color='skyblue')
plt.title('Exit Rate by Age Group', fontsize=15, fontweight='bold')
plt.xlabel('Age Group', fontsize=13)
plt.ylabel('Exit Count', fontsize=13)
plt.show()



-  People in their 50s have the highest churn rate of all customers, so we set them as our target audience.
- Conduct a deeper analysis of your target audience.
- When we looked at the data earlier, we saw a high percentage of data with a balance of 0, which we thought might affect our analysis, so we removed the data with a balance of 0 from our deeper analysis.

# 2. Detailed Plan Establishment <a class='anchor' id='ch2'></a>
- **Target group**: 50s & non-zero balance customers

In [ ]:
# Non-zero balance customers
data_ybal = data.loc[data['Balance'] != 0, :]
data.shape[0]
data_ybal.shape[0]

In [ ]:
# Generating a derived variable by Age
data_ybal['AgeGroup'] = pd.cut(data_ybal['Age'], bins = [10, 20, 30, 40, 50, 60, 70, 100],
                        labels=['10s', '20s', '30s', '40s', '50s', '60s', '70s up'],
                        include_lowest=True, right = False)
data_ybal.info()

In [ ]:
# Customers in their 50s
data_ybal_50 = data_ybal.loc[data_ybal['AgeGroup'] == '50s', :]

In [ ]:
# Drop 'AgeGroup' as all customers of data_ybal_50 are in their 50s
data_ybal_50.drop('AgeGroup', axis=1, inplace=True)

dy5 = data_ybal_50
dy5.info()

In [ ]:
!pip install pingouin

In [ ]:
# Correlation coefficient and p-value
import pingouin as pg
table = pg.rcorr(dy5, stars=False)
print(table)

In [ ]:
# Correlation visualization
corr = dy5[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']].corr()

import seaborn as sns
sns.heatmap(corr, vmin=-1, vmax=1, cmap='vlag', annot=True)

## 2.1 Deeper EDA<a class='anchor' id='ch2.1'></a>
- Conduct the deeper EDA focused on the target group we set

In [ ]:
dy5[['CreditScore', 'Age', 'Tenure', 'Balance','NumOfProducts', 'EstimatedSalary']].describe().T

In [ ]:
train['AgeGroup'].value_counts()

In [ ]:
dy5_0 = train.loc[(train['AgeGroup']=='50s') & (train['Balance']!=0)]

# Geographic distribution of 50 customers
plt.figure(figsize=(7, 4))
new_x_labels = ['Germany', 'France', 'Spain']
ax = sns.countplot(x='Geography', data=dy5_0, palette='cool')

plt.xlabel('Country', fontsize=13)
plt.xticks(range(len(new_x_labels)), new_x_labels)

plt.show()

- France is the most common overall customer, but Germany is the most common in their 50s.
- Germany also has the most customers in their 40s.
- However, the churn rate is also highest in Germany.
- France: Churn rate is about 48.6%, retention rate is about 51.4
- Germany: Churn rate is around 75.9%, retention rate is around 24.1
- Spain: bounce rate is around 51.5%, retention rate is around 48.5


In [ ]:
labels = ['Exited', 'Stayed']
france_sizes = [48.6, 51.4]
germany_sizes = [75.9, 24.1]
spain_sizes = [51.5, 48.5]
colors = ['#ff9999','#66b3ff']

# Generating pie chart
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Pie chart of France
axs[0].pie(france_sizes, labels=['Exited', 'NonExited'], colors=colors, autopct='%1.1f%%', startangle=140)
axs[0].set_title('France')

# Pie chart of Germany
axs[1].pie(germany_sizes, labels=['Exited', 'NonExited'], colors=colors, autopct='%1.1f%%', startangle=140)
axs[1].set_title('Germany')

# Pie chart of Spain
axs[2].pie(spain_sizes, labels=['Exited', 'NonExited'], colors=colors, autopct='%1.1f%%', startangle=140)
axs[2].set_title('Spain')

plt.tight_layout()
plt.show()

In [ ]:
# 50s Gender Distribution
plt.figure(figsize=(7, 4))
new_x_labels = ['Male', 'Female']
ax = sns.countplot(x='Gender', data=dy5_0, palette=['#6EACE6', '#FD78BB'])

plt.xlabel('Gender', fontsize=13)
plt.xticks(range(len(new_x_labels)), new_x_labels)
sns.set_style("whitegrid")
plt.show()

In [ ]:
# Credit score distribution in the 50s
plt.figure(figsize=(7, 4))
sns.histplot(dy5_0['CreditScore'], bins=30, color='#C8A4E2', edgecolor='#B477DD')

# Average line
plt.axvline(dy5_0['CreditScore'].mean(), color='#2989DF', linestyle='dashed', linewidth=2)

# Details
plt.xlabel('CreditScore', fontsize=13)
plt.grid(True)

plt.show()

- Not significantly different from the overall customer distribution

In [ ]:
# Tenure Distribution
plt.figure(figsize=(7, 4))
ax = sns.countplot(x='Tenure', data=dy5_0, palette='cool')
plt.xlabel('Tenure(year)', fontsize=13)
plt.grid(axis='y')
plt.show()

- Similarly, the overall customer distribution is similar.

In [ ]:
# Balance distribution for customers with balances
plt.figure(figsize=(7, 4))
sns.histplot(dy5_0['Balance'], bins=30, color='#735CE0', edgecolor='#735CE0')

plt.axvline(dy5_0['Balance'].mean(), color='#B6E7FD', linestyle='dashed', linewidth=2)
plt.xlabel('Balance', fontsize=13)
plt.grid(axis='y')
plt.show()

- The average balance is approximately $122,206.94

In [ ]:
# Number of products used
sns.countplot(x='NumOfProducts', data=dy5_0, palette='cool')

plt.grid(axis='y')
plt.xlabel('NumOfProducts', fontsize=13)
plt.show()

- Have an average of 1.3
- Compared to other customer segments, there are a lot of customers who only use one.
- Those in their 50s have 1.31, and those outside of their 50s have 1.57 (analyzed only among churners)

In [ ]:
# Whether customers have a credit card
card_counts = dy5_0['HasCrCard'].value_counts()

# Pie chart
plt.figure(figsize=(5, 5))
plt.pie(card_counts, labels=['Y', 'N'], autopct='%1.1f%%', colors=['skyblue', 'lightcoral'], startangle=140)
plt.title('HasCrCard', fontsize=15, fontweight='bold')
plt.show()

In [ ]:
data_nonzero_balance = dy5_0[dy5_0['Balance'] != 0]


plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
sns.histplot(data_nonzero_balance, x='CreditScore', hue='Exited', kde=True, multiple='stack', bins=30)
plt.title('CreditScore')
plt.subplot(2, 2, 2)
sns.histplot(data_nonzero_balance, x='Age', hue='Exited', kde=True, multiple='stack', bins=30)
plt.title('Age')
plt.subplot(2, 2, 3)
sns.histplot(data_nonzero_balance, x='Balance', hue='Exited', kde=True, multiple='stack', bins=30)
plt.title('Balance')
plt.subplot(2, 2, 4)
sns.histplot(data_nonzero_balance, x='Tenure', hue='Exited', kde=False, multiple='stack', bins=20)
plt.title('Tenure')


plt.tight_layout()
plt.show()

In [ ]:
# Visualize churn rate by account balance (Violin Plot)
plt.figure(figsize=(7, 4))
sns.violinplot(x='Exited', y='Balance', data=dy5_0, palette="muted", split=True)
plt.axhline(y=dy5_0['Balance'].median(), color='black', linestyle='--', label='Median Balance')
plt.title('Balance vs. Exited for Customers in their 50s', fontsize=15)
plt.xlabel('Exited', fontsize=12)
plt.ylabel('Balance', fontsize=12)
plt.xticks([0, 1], ['Not Exited', 'Exited'])
plt.legend()
plt.show()

In [ ]:
# Create a derivative variable
 # Derivative variable based on balance
import numpy as np
count1, bin_dividers1 = np.histogram(dy5['Balance'], bins=3)
a1 = np.linspace(dy5['Balance'].min(), dy5['Balance'].max(), 4)

dy5['BalLevel'] = pd.cut(dy5['Balance'], bins = bin_dividers1, include_lowest=True, labels=['BalLevel_Low','BalLevel_Mid', 'BalLevel_High'])
dy5.info()

# 3. Statistical Analysis <a class='anchor' id='ch3'></a>
- Performed a t-test for continuous independent variables and a chi-square test for categorical independent variables.

# 3.1 Sampled T-test<a class='anchor' id='ch3.1'></a>
- Frequency analysis of the dependent variable for each group shows a large difference in degrees, which affects the t-test.
- Proceed by sampling


- Test for differences in CreditScore to Balance in two populations with values of 0,1 in the Exited column.
- Remove 0 values in Balance
- 2Sample t-test is used for testing, ɑ=0.05
- Hypothesis
    - H0: The mean of the dependent variable between the Exited groups is the same.
    - H1: The mean of the dependent variable between the Exited groups is different.

In [ ]:
train_data = train[:]
nonzero=train_data[train_data['Balance']!=0].reset_index(drop=True)
df_0=nonzero.loc[nonzero['Exited']==0,['CreditScore','Age','Tenure','Balance']].reset_index()
df_1=nonzero.loc[nonzero['Exited']==1,['CreditScore','Age','Tenure','Balance']].reset_index()

In [ ]:
# CreditScore
import scipy.stats as stats
t, p=stats.ttest_ind(df_0.CreditScore,df_1.CreditScore)
t, p
if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

In [ ]:
# Age
t, p=stats.ttest_ind(df_0.Age,df_1.Age)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

In [ ]:
# Tenure
t, p=stats.ttest_ind(df_0.Tenure,df_1.Tenure)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

In [ ]:
# Balance
t, p=stats.ttest_ind(df_0.Balance,df_1.Balance)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")
print(round(p,5))

### 50s 2Sample T-test
- Deeper analysis by Exited group for those in their 50s with the highest churn rate
- Excludes values with a balance of 0
- No additional sampling because the number of samples is less than the previous experiment.
- Since the experiment is conducted on 50s, information on Age is included, so Age is not conducted.
- H0: The mean of the dependent variable is the same between the Exited groups.
- H1:Exited groups have different means for the dependent variable.


In [ ]:
def categorize_age(age):
    if age<20:
        return '10s'
    elif age<30:
        return '20s'
    elif age<40:
        return '30s'
    elif age<50:
        return '40s'
    elif age<60:
        return '50s'
    elif age<70:
        return '60s'
    else:
        return '70+'

train_data['Age_range']=train_data['Age'].apply(categorize_age)
nonzero=train_data[train_data['Balance']!=0].reset_index(drop=True)
df_0_50s=nonzero.loc[(nonzero['Exited']==0)&(nonzero['Age_range']=='50s'),['CreditScore','Tenure','Balance']].reset_index(drop=True)
df_1_50s=nonzero.loc[(nonzero['Exited']==1)&(nonzero['Age_range']=='50s'),['CreditScore','Tenure','Balance']].reset_index(drop=True)



In [ ]:
# CreditScore
t, p=stats.ttest_ind(df_0_50s.CreditScore,df_1_50s.CreditScore)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

In [ ]:
# Tenure
t, p=stats.ttest_ind(df_0_50s.Tenure,df_1_50s.Tenure)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

In [ ]:
# Balance
t, p=stats.ttest_ind(df_0_50s.Balance,df_1_50s.Balance)
t, p

if p < 0.05:
    print("Reject H0: There is a statistically significant difference between the two samples.")
else:
    print("Adopt H0: There is no statistically significant difference between the two samples.")

print(round(p,5))

### Conclusion
- Overall, we found statistically significant differences in CreditScore, Age, and Tenure, with the exception of Balance. However, a deeper analysis of the 50s, the age group with the highest churn rate, shows that there is no significant difference between CreditScore, Tenure, and Balance for the 50s alone, so it is likely that there are other variables or external factors that determine whether or not a 50-year-old churns.


# 3.2 Chi-Square Test<a class='anchor' id='ch3.2'></a>
- This is done to ascertain whether there exists a relationship between two categorical variables.
- **We also binned the Balance variable into BalLevel to see if creating a derivative variable from a continuous variable that the t-test found to be unrelated changed the results.**
- The list of categorical variables in this data:
    **HasCrCard, IsActiveMember, Gender, BalLevel, Exited**

- H0(Null Hypothesis): The two categorical variables are independent.
- H1(Alternative Hypothesis): The two categorical variables are not independent.


In [ ]:
# Preprocessing
# One-hot Encoding
dy5_dm = pd.get_dummies(dy5, columns = ['Gender','BalLevel'], drop_first=True)

- Credit card and active member are already dichotomous categorical variables, so we don't perform one-hot encoding.

In [ ]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(dy5_dm)
dy5_dm_sc = scaler.transform(dy5_dm)

dy5_dm_sc = pd.DataFrame(dy5_dm_sc)
dy5_dm_sc

dy5_dm_sc.columns = dy5_dm.columns

In [ ]:
col = ['HasCrCard', 'IsActiveMember', 'Gender_Male','BalLevel_Mid', 'BalLevel_High']
for i in col:
    cross_data = pd.crosstab(index = dy5_dm_sc['Exited'], columns = dy5_dm_sc[i],
                             margins=False)

    from scipy.stats import chi2_contingency
    chi2, p, dof, expected = chi2_contingency(cross_data)
    msg = 'Test Statistic: {}\np-value: {}\nDegree of Freedom: {}'
    print(f'<Exited and {i}>')
    print(msg.format(chi2, p, dof))
    if p < 0.05:
        print('dependent')
    else:
        print('independent')
    # print(expected)
    print('---------------------------')

- The following categorical variables were determined to be associated with the Exited variable based on the chi-square analysis.
    - IsActiveMember, Gender_Male, BalLevel
- Notably, the results of the t-test with Balance as a continuous variable were different from the results of the crosstabulation analysis with Balance as a categorical variable with bins.

# 4. Regression <a class='anchor' id='ch4'></a>
- In the preceding hypothesis testing step, it was verified that each variable did not exert a significant impact on the exit of customers in their 50s.
- In order to conduct regression analysis to apply what we've learned, we established a new continuous dependent variable called 'NumOfProducts'.
- Since 'Exited,' the primary subject of interest, is a categorical variable, it is emphasized that other testing methods such as the chi-square test should be employed.

In [ ]:
# Regression on NumOfProducts


X1 = dy5_dm_sc[['Age', 'BalLevel_Mid','BalLevel_High',
               'Tenure', 'CreditScore']]
y = dy5_dm_sc['NumOfProducts']

In [ ]:
# Check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif['variable'] = X1.columns
print(vif)

In [ ]:
import statsmodels.api as sm

X1_ad = sm.add_constant(X1)
model = sm.OLS(y, X1_ad).fit()
print(model.summary())

# 5. Conclusion<a class='anchor' id='ch5'></a>
- During the correlation visualization, the correlation coefficients between each variable were very low.
- The t-test showed that certain continuous variables did not have a significant impact on 50-something disengagement.
- The crosstabulation analysis showed that some categorical variables were influential.
- A new continuous dependent variable, NumOfProducts, was created to run the regression, but the model was not statistically significant, making it difficult to explain the relationship between each variable.

Banks are currently suffering from a large number of customer churn, resulting in a loss of trust and reduced revenues. In addition, the reduced likelihood of new product and event success due to a lack of active customers can have a significant impact on a bank's long-term growth strategy.
In particular, the loss of customers in the 50s and older, who tend to have higher balances and average annual salaries than other customers, can lead to a decrease in funding for the bank and a decrease in competitiveness.

Solutions include:

- Country-specific marketing
    - For example, offering cash back on beer for German customers.
    - Expected effect: Drive customer engagement, increase brand loyalty, and gain favorable perception against competitors.

- Leverage local flavor
    - You can organize a festival in collaboration with the local community.
    - Expected outcomes: Engage customers, strengthen brand image, build customer loyalty and long-term relationships

- Make banking affordable and easy for low-balance customers
    - Minimize the financial burden of low-balance customers due to their high churn rate.
    - Expected impact: Reduced financial burden, increased access to banking services, and attracted new customers

- Launch a product with higher interest on deposits
    - Reduce customer churn, increase retention, and attract new members by launching a product that offers higher interest on deposits, regardless of balance.
    - Expected impact: Reduced churn, increased retention, and an edge over competing banks.

- Offer retirement and pension management consulting
    - Tailor services for customers in their 50s and beyond to reduce churn among customers in their 50s, a group with a high churn rate, and attract new members.
    - Expected impact: Reducing customers' financial anxiety, building trust, increasing customer loyalty and bank credibility.

# 6. Project Evaluation <a class='anchor' id='ch6'></a>

- Disappointments
    - We did not establish a direction and plan for data analysis at the beginning, so it took a lot of time to analyze and verify various variables.

    - We selected 50 customers with high churn rate to analyze, but there is a regret that we did not find very close variables and focused too much on churn rate.

- Lessons learned
    - We liked the process of solving the project through collaboration and communication with our teammates, and it was a good growth process to review and understand what we learned.

    - Even though it was a small project, it was good to have this experience and it will help us in future projects.